# 1141_社會資料分析 作業四
學生：劉晏成

以「2018台北大學社會系三鶯調查」（brilliant_working.sav）檔案回答以下問題：
1. 以變異數分析方法（包括整體檢定與事後比較），檢視不同年齡分組者（agegroup）的收入（inc）是否有顯著差異。
2. 以教育年數（eduy）作為共變量，再次檢視檢視不同年齡分組者（agegroup）的收入（inc）是否有顯著差異。

In [1]:
from load import load_sav
import pandas as pd

df = load_sav("../data/brilliant_working.sav")

In [2]:
variable_value_labels = df.attrs['variable_value_labels']
column_names = df.attrs['column_names']
column_names_to_labels = df.attrs['column_names_to_labels']

## Q1: 以變異數分析方法（包括整體檢定與事後比較），檢視不同年齡分組者（agegroup）的收入（inc）是否有顯著差異。

In [3]:
df_1 = df[['agegroup', 'inc']].copy()

首先，先確認不同年齡組別分布

In [4]:
df_1['agegroup'].value_counts()

agegroup
1.0    336
2.0    327
3.0    205
Name: count, dtype: int64

確認共三組（1, 2, 3）

檢查缺漏值狀況

In [5]:
from statistic.missing_value import check_missing_value

check_missing_value(df_1)

,# of Missing Values
agegroup,0
inc,4


發現有缺漏值，預處理inc 為缺漏值的資料

In [6]:
df_1_cleared = df_1.dropna()
check_missing_value(df_1_cleared)

,# of Missing Values
agegroup,0
inc,0


執行ANOVA  
設定：不同年齡組別的收入平均數分別為 $\mu_1$、$\mu_2$、$\mu_3$  
$H_0: \mu_1 = \mu_2 = \mu_3$  
$H_1: \mu_k 不完全相等$  
$\alpha = 0.05$


In [7]:
from statistic.anova import perform_anova


perform_anova(df=df_1_cleared, val_col='inc', group_col='agegroup')

,sum_sq,df,F,PR(>F)
Intercept,4.037519e+11,1.0,454.077715,2.982257e-81
C(agegroup),2.568289e+10,2.0,14.442069,6.773292e-07
Residual,7.655747e+11,861.0,NaN,NaN


依據ANOVA 結果，agegroup 的P value < 0.05 ，拒絕虛無假設。結果顯示**三組間存在差異**

接下來執行Levene's test 來判斷均值性狀況，以執行事後比較

In [8]:
from scipy.stats import levene

groups = {name: group['inc'].values for name, group in df_1_cleared.groupby('agegroup')}

levene_stat, levene_p = levene(*groups.values())
pd.DataFrame({ 'Values': [f"{levene_stat:.2f}", levene_p] }, index = ['Levene Stat', 'P Value'])

,Values
Levene Stat,6.80
P Value,0.001176


結果顯示P Value 遠小於0.05：非均值。  
由於樣本數>50 (Shingala and Rajyaguru, 2015)，選擇games-howell 方法求得三組間兩兩比較的結果

In [9]:
from statistic.anova import perform_post_hoc

perform_post_hoc(df_1_cleared, levene_p = levene_p, val_col='inc', group_col='agegroup')

,A,B,mean(A),mean(B),diff,se,T,df,pval,hedges
0,1.0,2.0,34716.417910,42731.481481,-8015.063571,2268.694754,-3.532896,612.427945,0.001283,-0.276061
1,1.0,3.0,34716.417910,28829.268293,5887.149618,2648.795612,2.222576,356.062961,0.068779,0.208405
2,2.0,3.0,42731.481481,28829.268293,13902.213189,2884.143052,4.820223,434.151028,0.000006,0.429537


### Answer
由Anova 以及事後多重比較結果可得知：
1. 三組年齡組別的平均收入不完全相同
2. 年齡組別與收入關係為：2 > 1 > 3

## Q2: 以教育年數（eduy）作為共變量，再次檢視檢視不同年齡分組者（agegroup）的收入（inc）是否有顯著差異。

In [10]:
df_2 = df[['eduy', 'agegroup', 'inc']].copy()

檢查缺漏值狀況

In [11]:
from statistic.missing_value import check_missing_value

check_missing_value(df_2)

,# of Missing Values
eduy,0
agegroup,0
inc,4


處理inc 缺漏值

In [12]:
df_2_cleared = df_2.dropna()
check_missing_value(df_2_cleared)

,# of Missing Values
eduy,0
agegroup,0
inc,0


基於ANCOVA 的兩個假設做假設檢驗
1. **共變量與因變量**需為線性關係
2. **共變量與自變量（組別）間** 不得有交互作用

首先檢驗共變數與因變數是否為線性關係

In [13]:
from statistic.anova import check_linearity

check_linearity(df_2_cleared, val_col='inc', covar_col='eduy')

,sum_sq,df,F,PR(>F)
Intercept,4.316321e+09,1.0,5.029773,2.516859e-02
eduy,5.152873e+10,1.0,60.046009,2.603274e-14
Residual,7.397289e+11,862.0,NaN,NaN


由eduy 的P value 可知教育年數與收入符合線性關係。

往下繼續檢查共變數與自變數的互動關係：檢驗迴歸同質性

In [14]:
from statistic.anova import check_interaction

check_interaction(df_2_cleared, val_col='inc', group_col='agegroup', covar_col='eduy')

,sum_sq,df,F,PR(>F)
Intercept,1.078969e+10,1.0,13.254101,0.000288
C(agegroup),1.420332e+10,2.0,8.723707,0.000178
eduy,1.733675e+08,1.0,0.212965,0.644570
C(agegroup):eduy,1.877668e+10,2.0,11.532675,0.000011
Residual,6.984674e+11,858.0,NaN,NaN


由agegroup: eduy 的P value > 0.05 可知年齡分組與教育程度交互作用顯著，故不符合ANCOVA 的假設，不繼續進行ANCOVA。

### Answer

教育程度在不同年齡分組對於因變量——收入有不同的影響力，停止分析。